In [29]:
# ! pip install openai
# ! pip install google
# ! pip install llama-index-readers-web
# ! pip install llama-index-llms-openai
# %pip install llama-index-program-openai
# %pip install llama-index-llms-llama-api
! pip install llama-index-embeddings-openai

In [46]:
import os
from openai import OpenAI as open_ai
import re
import numpy as np
os.environ["OPENAI_API_KEY"] = 'sk-proj-YKwxWCpNeC8vuhRwigU9T3BlbkFJqpNo6wxzV5dZyRmEzWO4'
client = open_ai()

In [106]:
setup_string = """
Your job is to return Google search queries, specifically Google Dorks, to find more
about a subject given a dumped text file of all of the information we currently know
about the subject. Not all of this information should be used in every query.
You can output up to twenty queries, and they should be formatted in a bullet point list
format, where every bullet point contains only the Google Dork search query and nothing
else (without quotes).

The end goal is to create an OSINT (open source intelligence) report about
the given subject. For example, if the user wants to find more about a person,
you would help them find out about their location, age, education, associated
institutions, relevant connections, and so forth.
""" # add osint flowchart

setup_string_two = """
The end goal is to create an OSINT (open source intelligence) report about
the given subject. For example, if the user wants to find more about a person,
you would help them find out about their location, age, education, associated
institutions, relevant connections, and so forth.
""" # add osint flowchart

In [7]:
prompt = """
I want to find out more about the person Jai Sharma using Google Dorks. Here are
some things I know about him:
- Went to Monta Vista High School
- Is friends with Milind Maiti
- Might be associated with Berkeley
- Research oriented and possibly interested in Machine Learning, wrote a paper with Christopher Sun
"""

In [16]:
# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": setup_string},
#     {"role": "user", "content": prompt}
#   ]
# )

# result = completion.choices[0].message.content
# search_list = result.split("\n")
# search_list = [i[2:] for i in search_list]
# print(result)
# print(search_list)

- "Jai Sharma" "Monta Vista High School" "Berkeley"
- "Jai Sharma" "Milind Maiti"
- "Jai Sharma" "Machine Learning" "Christopher Sun"
- "Jai Sharma" "research" "Berkeley"
- "Jai Sharma" "publication" "Christopher Sun"
- "Jai Sharma" "Machine Learning" site:berkeley.edu
- "Jai Sharma" "Monta Vista High School" "alumni"
- "Jai Sharma" "research group" "Berkeley"
- "Jai Sharma" "Christopher Sun" "co-author"
- "Jai Sharma" "Machine Learning" "publication" -site:github.com
- "Jai Sharma" "research interests" "Berkeley"
- "Jai Sharma" "Monta Vista High School" "facebook"
['"Jai Sharma" "Monta Vista High School" "Berkeley"', '"Jai Sharma" "Milind Maiti"', '"Jai Sharma" "Machine Learning" "Christopher Sun"', '"Jai Sharma" "research" "Berkeley"', '"Jai Sharma" "publication" "Christopher Sun"', '"Jai Sharma" "Machine Learning" site:berkeley.edu', '"Jai Sharma" "Monta Vista High School" "alumni"', '"Jai Sharma" "research group" "Berkeley"', '"Jai Sharma" "Christopher Sun" "co-author"', '"Jai Shar

In [17]:
# from googlesearch import search

# # query = "Geeksforgeeks"
# urls = []

# for query in search_list:
#   for j in search(query, tld="co.in", num=10, stop=10, pause=2):
#       urls.append(j)
#       print(j)

In [18]:
# urls

['https://www.linkedin.com/in/rithwik-nukala',
 'https://www.academia.edu/103814896/Exploring_Algebraic_Conditions_on_Counterexamples_of_the_Collatz_Conjecture',
 'https://www.linkedin.com/in/nithyaappannagaari',
 'https://www.instagram.com/mvresearchclub/',
 'https://ieeexplore.ieee.org/iel7/9987717/9987755/09988121.pdf',
 'https://arxiv.org/abs/2108.06628',
 'https://www.researchgate.net/scientific-contributions/Milind-Maiti-2199726740',
 'https://vixra.org/author/jai_sharma',
 'https://vixra.org/author/milind_maiti',
 'https://arxiv.org/abs/2205.11230',
 'https://www.researchgate.net/scientific-contributions/Jai-Sharma-2199738343',
 'https://www.preprints.org/manuscript/202207.0097/v1',
 'https://www.researchsquare.com/article/rs-1769921/v1.pdf',
 'https://www.researcher-app.com/paper/7041375',
 'https://arxiv.org/abs/2108.06628',
 'https://www.researchgate.net/scientific-contributions/Jai-Sharma-2199738343',
 'https://www.researchgate.net/publication/363724723_Leveraging_Machine_Le

In [97]:
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core import SummaryIndex, Document
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.node_parser import HTMLNodeParser
from llama_index.llms.openai import OpenAI
from llama_index.llms.llama_api import LlamaAPI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.embeddings.openai import OpenAIEmbedding

# # urls = ["https://en.wikipedia.org/wiki/Eminem"]
# documents = SimpleWebPageReader().load_data(urls[:5])

# for i in range(len(documents)):
#   documents[i].text = re.sub(r'\\s+', ' ', documents[i].text)
#   documents[i].text = re.sub(r' +', ' ', documents[i].text)
#   documents[i].text = re.sub(r'\\n', '\n', documents[i].text)
#   documents[i].text = re.sub(r'\n +', '\n', documents[i].text)
#   documents[i].text = re.sub(r'\n+', '\n', documents[i].text)
#   print(documents[i].text)

# parser = HTMLNodeParser(
#     # tags=["p", "h1"],
#     chunk_lines=100,
#     chunk_lines_overlap=50,
#     max_chars=1500,
#     )

# nodes = parser.get_nodes_from_documents(documents)
# nodes = [node for node in nodes if len(node.text) > 0]
# for node in nodes:
#   print(node.text)
#   print(node.metadata)

In [22]:
# print(nodes[2].text)

In [23]:
# len(nodes)

12

In [30]:
# index = VectorStoreIndex(nodes=nodes)

In [25]:
# Settings.llm = client # OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [26]:
# query_engine = index.as_query_engine()
# response = query_engine.query(prompt)
# print(response)

Jai Sharma may have attended Monta Vista High School, is acquainted with Milind Maiti, could have ties to Berkeley, and is likely research-oriented with a potential interest in Machine Learning, having collaborated on a paper with Christopher Sun.


In [36]:
# retriever = VectorIndexRetriever(
#     index=index,
#     similarity_top_k=10,
# )

# # # configure response synthesizer
# # response_synthesizer = get_response_synthesizer()

# # # assemble query engine
# # query_engine = RetrieverQueryEngine(
# #     retriever=retriever,
# #     response_synthesizer=response_synthesizer,
# #     node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
# # )

# # # query
# # response = query_engine.query(prompt)
# # print(response)

# relevant_nodes = retriever.retrieve(prompt)

In [41]:
added_information = "\n\n\n".join([i.text for i in relevant_nodes])

RELATED PAPERS


About
Press
Blog
People
Papers
Topics
We're Hiring!
Help Center
Find new research papers in:
Physics
Chemistry
Biology
Health Sciences
Ecology
Earth Sciences
Cognitive Science
Mathematics
Computer Science
Terms
Privacy
Copyright
Academia ©2024


Related Papers


Log In
Sign Up
more
AboutPressBlogPeoplePapersTermsPrivacyCopyrightWe're Hiring!Help Centerless
About
Press
Blog
People
Papers
Terms
Privacy
Copyright
We're Hiring!
Help Center
less


International Journal of Mathematics and Mathematical Sciences
Peter Shiue
Samuel Bonaya Buya
arXiv: Combinatorics
2016 •
Maya Ahmed
Vicente Padilla
We confirm the convergence of the number 2 ^ 30402457 -1 To the Collatz conjecture
2024 •
Vasilios Andritsos
Ismael T Fortunado
arXiv (Cornell University)
2020 •
Anant Gupta
arXiv: Dynamical Systems
2019 •
Pablo Castaneda
Maruszka E M Meinard
Walter Carnielli
2023 •
Ahmad f. Alabbasi
Arxiv preprint arXiv:0804.3716
2008 •
Roupam Ghosh
Acta Informatica
2000 •
Stefan Andrei
Michael A Ido

In [98]:
Settings.llm = OpenAI(api_key=os.environ["OPENAI_API_KEY"], model="gpt-4o")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large", embed_batch_size=100)

In [187]:
def iteration(prompt):
  # Find dorks
  completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {"role": "system", "content": setup_string},
      {"role": "user", "content": prompt}
    ]
  )

  result = completion.choices[0].message.content
  search_list = result.split("\n")
  search_list = [i[2:] for i in search_list]
  print(search_list)
  # print(result)
  # print(search_list)

  urls = []
  for query in search_list:
    for j in search(query, tld="co.in", num=10, stop=10, pause=2):
        urls.append(j)
        print(j)

  np.random.seed(0)
  # url_shortlist = np.random.choice(urls, min(20, len(urls))).tolist()
  url_shortlist = urls
  # print(url_shortlist)
  documents = SimpleWebPageReader().load_data(url_shortlist)
  print(len(documents))
  for i in range(len(documents)):
    documents[i].text = re.sub(r'\\s+', ' ', documents[i].text)
    documents[i].text = re.sub(r' +', ' ', documents[i].text)
    documents[i].text = re.sub(r'\\n', '\n', documents[i].text)
    documents[i].text = re.sub(r'\n +', '\n', documents[i].text)
    documents[i].text = re.sub(r'\n+', '\n', documents[i].text)
    # print(documents[i].text)

  parser = HTMLNodeParser(
      tags=["p"], #"h1"],
      chunk_lines=100,
      chunk_lines_overlap=50,
      # max_chars=150,
      # chunk_size=100,
      # chunk_overlap=25
      # max_tokens=150
      )

  min_node_len = 0
  nodes = parser.get_nodes_from_documents(documents)
  nodes = [node for node in nodes if len(node.text) > min_node_len]
  # for node in nodes:
  #   if len(node.text) > 1500:
  #     node.text = node.text[:1500]
  for node in nodes:
    print(node.text)
    print(node.metadata)

  index = VectorStoreIndex(nodes=nodes)

  retriever = VectorIndexRetriever(
      index=index,
      similarity_top_k=10,
  )

  relevant_nodes = retriever.retrieve("System Role: " + setup_string_two + "\n\n\n" + "User Query: " + prompt)
  print("THE RELEVANT NODES: ")
  added_information = "\n\n\n".join([i.text for i in relevant_nodes])

  result = prompt + added_information

  return result, nodes

In [188]:
result, nodes = iteration(prompt)

['"Jai Sharma" "Monta Vista High School"', '"Jai Sharma" "Milind Maiti"', '"Jai Sharma" Berkeley', '"Jai Sharma" "Machine Learning"', '"Jai Sharma" "Christopher Sun" paper', '"Jai Sharma" CV filetype:pdf', '"Jai Sharma" "LinkedIn"', '"Jai Sharma" "GitHub"', '"Jai Sharma" "research paper"', '"Jai Sharma" "machine learning" site:scholar.google.com', '"Jai Sharma" "research" site:berkeley.edu', '"Jai Sharma" "research interests"', '"Jai Sharma" "publications"', '"Jai Sharma" "conference" "machine learning"', '"Jai Sharma" "friends with Milind Maiti"', '"Jai Sharma" "email address"', '"Jai Sharma" "profile" site:facebook.com', '"Jai Sharma" "profile" site:twitter.com', '"Jai Sharma" "profile" site:instagram.com', '"Jai Sharma" "Monta Vista High School" "alumni"']
https://www.calibercircle.org/about/the-team
https://www.maxpreps.com/ca/cupertino/monta-vista-matadors/athletes/jai-sharma/bio/?careerid=72ip7tlqmns99
https://www.stemboost.org/instructors.html
https://ieeexplore.ieee.org/iel7/99

ConnectionError: HTTPSConnectionPool(host='www.localhost', port=443): Max retries exceeded with url: /herosteels.com/404.php (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d6522c42620>: Failed to resolve 'www.localhost' ([Errno -2] Name or service not known)"))

In [185]:
print(result)


I want to find out more about the person Jai Sharma using Google Dorks. Here are 
some things I know about him:
- Went to Monta Vista High School
- Is friends with Milind Maiti
- Might be associated with Berkeley
- Research oriented and possibly interested in Machine Learning, wrote a paper with Christopher Sun
Presentation on AI
Read less


Tomorrow, Lausanne welcomes seven Lynx alumni back to campus for the annual Young Alumni panel, an insightful session tailored for Upper School students.
Representing graduating classes from 2019 to 2023, these alumni will share valuable resources and the experiences that led to their choices in colleges, universities, graduate programs, internships and careers.
During the panel, Upper School students will gain helpful tools for their own academic and professional journeys as our alumni discuss how Lausanne equipped them for the future and offer practical advice for college-bound students. Panelists will also touch on the realities of college life

In [60]:
result = iteration(result)

https://www.linkedin.com/in/rithwik-nukala
https://www.linkedin.com/in/nithyaappannagaari
https://cdn.web.uta.edu/-/media/project/website/commencement/programs/2021/spring/commprog-inside_spring2021.ashx
https://www.middlesex.mass.edu/templateinframes/?Site=Help&Url=https://xn----8sbwhcgscfiy8e.xn--p1ai/k789ejnpam4r3v53xy275comamafe85
https://m.facebook.com/groups/sanctuaryasia/posts/10157689145591103/
https://m.facebook.com/groups/TIGERSofCENTRALINDIA/permalink/2999473583639142/
https://www.instagram.com/p/CB5ceIzDpN-/
https://www.instagram.com/p/CToWiwNvMOp/


In [125]:
print(result)


I want to find out more about the person Jai Sharma using Google Dorks. Here are 
some things I know about him:
- Went to Monta Vista High School
- Is friends with Milind Maiti
- Might be associated with Berkeley
- Research oriented and possibly interested in Machine Learning, wrote a paper with Christopher Sun
Jai Sharma is a junior at the University of California, Berkeley, studying Computer Science and Political Science, interested in international relations and political theory. Jai has worked as a research assistant to Professor Aila Matanock since January 2021 on various projects. He has also interned at Oracle as a database engineer. Outside of his academic interests, Jai enjoys collecting records and playing guitar.
Institute of International Studies

215 Philosophy Hall

Berkeley, CA 94720-2308
Email:
iis@berkeley.edu

Phone: (510)
642-1106

Fax: (510) 642-9493
Berkeley Main Page

Campus Administration

Berkeley News

The Campaign for Berkeley

Visiting Berkeley

Campus Event

In [69]:
nodes[4].text

'Exploring Algebraic Conditions on Counterexamples of the Collatz Conjecture\nExploring Algebraic Conditions on Counterexamples of the Collatz Conjecture'

In [113]:
[len(i.text) for i in nodes]

[174, 6, 76, 181, 151, 777, 14, 4657, 14, 844, 14, 243]

In [174]:
nodes[2].text

IndexError: list index out of range

In [182]:
print(nodes[0].text)

Jai Sharma is a junior at the University of California, Berkeley, studying Computer Science and Political Science, interested in international relations and political theory. Jai has worked as a research assistant to Professor Aila Matanock since January 2021 on various projects. He has also interned at Oracle as a database engineer. Outside of his academic interests, Jai enjoys collecting records and playing guitar.
Institute of International Studies

215 Philosophy Hall

Berkeley, CA 94720-2308
Email:
iis@berkeley.edu

Phone: (510)
642-1106

Fax: (510) 642-9493
Berkeley Main Page

Campus Administration

Berkeley News

The Campaign for Berkeley

Visiting Berkeley

Campus Events
Powered by Open Berkeley
Accessibility Statement
Nondiscrimination Policy Statement
Privacy Statement
Copyright © 2024 UC Regents; all rights reserved
Back to Top


In [178]:
len(nodes)

3